In [1]:
from IsingLib import Model, Formula, DaFormat
from pyqubo import Array, Sum
import numpy as np

np.set_printoptions(threshold=30)
np.set_printoptions(edgeitems=20)
np.set_printoptions(linewidth=200)

In [2]:
city = 5
vartype = 'BINARY'
x = Array.create('x', (city, city), vartype)
dis_li = [[0, 10, 8, 8, 4], [10, 0, 6, 11, 7], [8, 6, 0, 14, 8], [9, 11, 14, 0, 6], [4, 7, 8, 6, 0]]
distance = np.zeros((city**2, city**2))
for i in range(city):
    point = (i + 1) % city
    distance[i*city: (i+1)*city, point*city: (point+1)*city] = dis_li
distance = distance + distance.T
time_const = Sum(0, city, lambda i: (Sum(0, city, lambda j: x[i, j]) - 1) ** 2)
city_const = Sum(0, city, lambda j: (Sum(0, city, lambda i: x[i, j]) - 1) ** 2)

In [3]:
const_model = Model.create(vartype, time_const+city_const)
aim_model = Model(vartype, distance, symbol='x', shape=(city,city))

In [4]:
# print(aim_model)

In [5]:
h = Formula(aim=aim_model, const=const_model)

In [6]:
print(h)

H = {'aim': <IsingLib.model.Model object at 0x1020b8ef0>, 'const': <IsingLib.model.Model object at 0x155748278>}


In [7]:
model = h.compile(const=11)
print(model.const)

110.0


In [8]:
ans = model.solve('sa')

In [9]:
print(ans.states)
print(ans.enegries)

[array([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0])]
[35.0]


In [10]:
model.energy(ans.states[0])

35.0

In [11]:
for state in ans.states:
    print(h.check_constraints(state, 'const'))

0.0


In [12]:
# import itertools

# for v in itertools.permutations(range(city)):
#     d = 0
#     for i in range(city):
#         d += dis_li[v[i-1]][v[i]]
#     print(d, v)

In [13]:
da_model = model.to_da()
da = DaFormat(**da_model, api_key='../../da_key_2.json', version=2)
job_id = da.da_solver()
da.get_job(job_id)

In [14]:
da.get_energy(job_id)

array([-75, -75, -75, -75, -75, -75, -75, -75, -75, -75, -75, -75, -75, -75, -75])

In [15]:
state_li = da.get_configuration(job_id)
for state in state_li:
    print(state)

{'x[0][0]': 0, 'x[0][1]': 0, 'x[0][2]': 0, 'x[0][3]': 0, 'x[0][4]': 1, 'x[1][0]': 0, 'x[1][1]': 0, 'x[1][2]': 0, 'x[1][3]': 1, 'x[1][4]': 0, 'x[2][0]': 0, 'x[2][1]': 1, 'x[2][2]': 0, 'x[2][3]': 0, 'x[2][4]': 0, 'x[3][0]': 0, 'x[3][1]': 0, 'x[3][2]': 1, 'x[3][3]': 0, 'x[3][4]': 0, 'x[4][0]': 1, 'x[4][1]': 0, 'x[4][2]': 0, 'x[4][3]': 0, 'x[4][4]': 0}
{'x[0][0]': 0, 'x[0][1]': 0, 'x[0][2]': 0, 'x[0][3]': 0, 'x[0][4]': 1, 'x[1][0]': 0, 'x[1][1]': 1, 'x[1][2]': 0, 'x[1][3]': 0, 'x[1][4]': 0, 'x[2][0]': 0, 'x[2][1]': 0, 'x[2][2]': 1, 'x[2][3]': 0, 'x[2][4]': 0, 'x[3][0]': 1, 'x[3][1]': 0, 'x[3][2]': 0, 'x[3][3]': 0, 'x[3][4]': 0, 'x[4][0]': 0, 'x[4][1]': 0, 'x[4][2]': 0, 'x[4][3]': 1, 'x[4][4]': 0}
{'x[0][0]': 0, 'x[0][1]': 0, 'x[0][2]': 0, 'x[0][3]': 0, 'x[0][4]': 1, 'x[1][0]': 1, 'x[1][1]': 0, 'x[1][2]': 0, 'x[1][3]': 0, 'x[1][4]': 0, 'x[2][0]': 0, 'x[2][1]': 0, 'x[2][2]': 1, 'x[2][3]': 0, 'x[2][4]': 0, 'x[3][0]': 0, 'x[3][1]': 1, 'x[3][2]': 0, 'x[3][3]': 0, 'x[3][4]': 0, 'x[4][0]': 0, 'x[

In [20]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

model_dwave = h.compile(const=12)
bqm = model_dwave.to_bqm()
# print(tsp_dwave.to_ising())
print(bqm)

sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample(bqm)
print(response.record[0][0].reshape(city, city))

BinaryQuadraticModel({0: -24.0, 1: -24.0, 2: -24.0, 3: -24.0, 4: -24.0, 5: -24.0, 6: -24.0, 7: -24.0, 8: -24.0, 9: -24.0, 10: -24.0, 15: -24.0, 20: -24.0, 21: -24.0, 22: -24.0, 23: -24.0, 24: -24.0, 11: -24.0, 16: -24.0, 12: -24.0, 17: -24.0, 13: -24.0, 18: -24.0, 14: -24.0, 19: -24.0}, {(0, 1): 24.0, (0, 2): 24.0, (0, 3): 24.0, (0, 4): 24.0, (0, 5): 24.0, (0, 6): 10.0, (0, 7): 8.0, (0, 8): 8.0, (0, 9): 4.0, (0, 10): 24.0, (0, 15): 24.0, (0, 20): 24.0, (0, 21): 10.0, (0, 22): 8.0, (0, 23): 9.0, (0, 24): 4.0, (1, 2): 24.0, (1, 3): 24.0, (1, 4): 24.0, (1, 5): 10.0, (1, 6): 24.0, (1, 7): 6.0, (1, 8): 11.0, (1, 9): 7.0, (1, 11): 24.0, (1, 16): 24.0, (1, 20): 10.0, (1, 21): 24.0, (1, 22): 6.0, (1, 23): 11.0, (1, 24): 7.0, (2, 3): 24.0, (2, 4): 24.0, (2, 5): 8.0, (2, 6): 6.0, (2, 7): 24.0, (2, 8): 14.0, (2, 9): 8.0, (2, 12): 24.0, (2, 17): 24.0, (2, 20): 8.0, (2, 21): 6.0, (2, 22): 24.0, (2, 23): 14.0, (2, 24): 8.0, (3, 4): 24.0, (3, 5): 9.0, (3, 6): 11.0, (3, 7): 14.0, (3, 8): 24.0, (3, 9):